# Méthode affine primale

In [ ]:
using LinearAlgebra
using JuMP, HiGHS
using Plots

Considérons le problème jouet suivant.

In [ ]:
m = Model(HiGHS.Optimizer)
@variable(m, x[1:2] >= 0)
@constraint(m, c1, x[1]+x[2] <= 2)
@constraint(m, c2, -x[1]+x[2] <= 1)
@objective(m, Max, x[1] + 2x[2])

print(m)

In [ ]:
optimize!(m)

In [ ]:
value.(x)

In [ ]:
λ = -[dual(c1) ; dual(c2) ] # Nous prenons l'opposé comme il s'agit d'un problème de maximisation. 

Implémentons l'algorithme affine primal.

In [ ]:
mutable struct interiorPoint 
    
    k:: Int64
    K:: Int64
    
    x:: Vector  # current solution
    
    verbose
    trace
end

primalAffine = interiorPoint(0, 100, [], false, [])

In [ ]:
function (primalAffine:: interiorPoint)(x0:: Vector, c:: Vector, A:: Matrix, b:: Vector,
                                        ϵ:: Float64 = 1e-6)

    primalAffine.x = copy(x0)
    n = length(primalAffine.x)

    obj = dot(c,primalAffine.x)
    Δc = abs(obj)+2
    Δx = Inf*ones(n)

    if (trace)
        primalAffine.trace = [obj ; x0; zeros(n)]
        s = zeros(n)
    end
    
    γ = 0.995

    k = 0   # iteration index
    K = 100 # maximum number of iterations
    
    while ((Δc > ϵ*max(abs(obj),1)) && (k < K))
        k += 1

        X = diagm(primalAffine.x)
        # Δx := −(I − XAt(AX2At)−1AX)Xc;
        B = A*X
        Δx = -(I-B'*((B*B')\B))*X*c

        if (trace)
            s = diagm((primalAffine.x).^(-1))*Δx
            primalAffine.trace[n+2:2*n+1,k] = s
        end
    
        i = 0
        xi = 0
        for j = 1:n
            if -Δx[j] > xi
                xi = -Δx[j]
                i = j
            end
        end
        if (i == 0)
            println("Problème non borné!")
            break;
        end
        α = γ/xi
        newx = primalAffine.x+α*X*Δx
        newobj = dot(c, newx)
        Δc = obj-newobj
        obj = newobj
        primalAffine.x = newx

        if (trace)
            primalAffine.trace = hcat(primalAffine.trace, [obj ; newx; zeros(n)])
        end

    end
    
    primalAffine.k = k
    if (trace)
        # For simplicity, to avoid additional computations, we simply copy the last value of s
        primalAffine.trace[n+2:2*n+1,k+1] = s

        return (primalAffine.x).*s
    end

end

## Exemple

In [ ]:
c = [-1; -2; 0; 0]
A = [1 1 1 0; -1 1 0 1]
b = [2; 1]

In [ ]:
# Point initial
x0 = [0.5; 0.5; 1; 1]
trace = true

slack = primalAffine(x0, c, A, b)

println(slack)

In [ ]:
primalAffine.trace

In [ ]:
using DelimitedFiles

open("trace.txt", "w") do f
    writedlm(f, primalAffine.trace)
end

In [ ]:
k = primalAffine.k

In [ ]:
default(size=(600,600), fc=:heat)
x, y = 0:0.02:2.0, 0:0.02:2.0
z = Surface((x,y)->dot(c[1:2],[x,y]), x, y)
Plots.contour(x,y,z, linealpha = 0.2, levels=1000)

plot!(xlims = (0.0,  2.0), ylims = (0.0, 2.0))
plot!(x -> 2-x, color = :red, linewidth = 2, label = "Contrainte 1")
plot!(x -> 1+x, color = :blue, linewidth = 2, label = "Contrainte 2")
plot!(primalAffine.trace[2,:], primalAffine.trace[3,:], lw = 3, color = :green)
scatter!(primalAffine.trace[2,:], primalAffine.trace[3,:], markersize = 4, color = :grey, label = "Itérés")

In [ ]:
iterprimal = primalAffine.trace[2:5,:]
iterdual = primalAffine.trace[6:9,:]
slackness = zeros(4,k+1)
for i = 1:k+1
    slackness[:,i] = iterprimal[:,i].*iterdual[:,i]
end
slackness

Regardons l'évolution des écarts de complémentarité.

In [ ]:
plot([ norm(slackness[:,i]) for i = 1:8 ], label = "Slackness")

## Exemple 2: le problème du fermier

Problème donné dans Birge et Louveaux, "Introduction to Stochastic Programming", chapitre 1.

In [ ]:
c = [ 150 ; 230 ; 260 ; 238 ; 210 ; -170 ; -150 ; -36 ; -10; 0 ; 0 ; 0 ; 0 ; 0]
A = [ 1 1 1 0 0 0 0 0 0 1 0 0 0 0 ;
      2.5 0 0 1 0 -1 0 0 0 0 -1 0 0 0 ;
      0 3 0 0 1 0 -1 0 0 0 0 -1 0 0 ;
      0 0 -20 0 0 0 0 1 1 0 0 0 1 0;
      0 0 0 0 0 0 0 1 0 0 0 0 0 1]
b = [ 500 ; 200 ; 240 ; 0 ; 6000 ]

In [ ]:
x0 = [ 100 ; 100 ; 100 ; 50 ; 50 ; 50 ; 50 ; 500 ; 500 ]

In [ ]:
A[:,1:9]*x0-b

In [ ]:
x0 = [ x0 ; 200 ; 50 ; 60 ; 1000; 5500]

In [ ]:
A*x0

In [ ]:
slack = primalAffine(x0, c, A, b)

println(slack)

In [ ]:
primalAffine.x

In [ ]:
primalAffine.k